# Khipus.ai
## AI Agents
### Case Study: SQL Database Agent
<span>© Copyright Notice 2025, Khipus.ai - All Rights Reserved.</span>


## Setup

In [1]:
import os
from IPython.display import Markdown, HTML, display
import openai
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool
from typing import List, Dict, Any, Optional
import json

## data to the SQL database

In [4]:
import pandas as pd
# Path to your SQLite database file
database_file_path = "./db/test.db"

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
#engine = create_engine(f'sqlite:///{database_file_path}')
#file_url = "./data/sales_data.csv"
#df = pd.read_csv(file_url).fillna(value = 0)
#df.to_sql(
#    'sales_data',
#    con=engine,
#    if_exists='replace',
#    index=False
#)

In [5]:
# Connect to the SQLite database using the path already defined earlier
import sqlite3

conn = sqlite3.connect(database_file_path)

# Query to get data from the sales_data table with a limit
query = "SELECT * FROM sales_data LIMIT 10;"
sales_data = pd.read_sql_query(query, conn)

# Display the total number of rows and a sample of the data
print(f"Total rows in sales_data: {pd.read_sql_query('SELECT COUNT(*) FROM sales_data', conn).iloc[0, 0]}")
display(sales_data)

Total rows in sales_data: 2823


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,0,NYC,NY,10022,USA,0,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de lAbbaye,0,Reims,0,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,0,Paris,0,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,0,Pasadena,CA,90003,USA,0,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,0,San Francisco,CA,0,USA,0,Brown,Julie,Medium
5,10168,36,96.66,1,3479.76,10/28/2003 0:00,Shipped,4,10,2003,...,9408 Furth Circle,0,Burlingame,CA,94217,USA,0,Hirano,Juri,Medium
6,10180,29,86.13,9,2497.77,11/11/2003 0:00,Shipped,4,11,2003,...,"184, chausse de Tournai",0,Lille,0,59000,France,EMEA,Rance,Martine,Small
7,10188,48,100.00,1,5512.32,11/18/2003 0:00,Shipped,4,11,2003,...,"Drammen 121, PR 744 Sentrum",0,Bergen,0,N 5804,Norway,EMEA,Oeztan,Veysel,Medium
8,10201,22,98.57,2,2168.54,12/1/2003 0:00,Shipped,4,12,2003,...,5557 North Pendale Street,0,San Francisco,CA,0,USA,0,Murphy,Julie,Small
9,10211,41,100.00,14,4708.44,1/15/2004 0:00,Shipped,1,1,2004,...,"25, rue Lauriston",0,Paris,0,75016,France,EMEA,Perrier,Dominique,Medium


## Configure Azure OpenAI Client

In [ ]:
# Configure Azure OpenAI client
#endpoint = os.getenv("ENDPOINT_URL", "https://your-resource.openai.azure.com/")
#deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
#api_key = os.getenv("AZURE_OPENAI_API_KEY", "your-api-key-here")
#api_version = "2024-10-01-preview"


# ----- Azure OpenAI Configuration -----
endpoint = os.getenv("ENDPOINT_URL", "https://khipusaigpt0566189501.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_key = os.getenv("AZURE_OPENAI_API_KEY", "Your key here") # 5vcXkwMqlTG134VpmU85v0Ob37TSYuQ3hIoHg3ROy48W0Xu4EOy9JQQJ99BEACHYHv6XJ3w3AAAAACOGhtnQ
api_version = "2025-01-01-preview"
client = openai.AsyncAzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=api_key,
)

MODEL_NAME = deployment
print(f"✅ Azure OpenAI configured with model: {MODEL_NAME}")

✅ Azure OpenAI configured with model: gpt-4o


## Define Database Tools for the Agent

In [8]:
def _get_db_connection():
    """Get a database connection to the SQLite database."""
    return sqlite3.connect(database_file_path)


@function_tool
def get_table_list() -> List[str]:
    """Return a list of all tables in the database."""
    conn = _get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
    tables = [row[0] for row in cursor.fetchall()]
    conn.close()
    return tables


@function_tool
def get_table_schema(table_name: str) -> List[Dict[str, Any]]:
    """Return the schema (column information) for a specific table."""
    conn = _get_db_connection()
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    schema = []
    for row in cursor.fetchall():
        schema.append({
            "column_id": row[0],
            "name": row[1], 
            "type": row[2],
            "not_null": bool(row[3]),
            "default_value": row[4],
            "primary_key": bool(row[5])
        })
    conn.close()
    return schema


@function_tool
def execute_sql_query(sql_query: str, limit: int = 10) -> Dict[str, Any]:
    """Execute a SQL query and return the results. Limited to SELECT statements only for safety."""
    # Safety check - only allow SELECT queries
    if not sql_query.strip().upper().startswith('SELECT'):
        return {"error": "Only SELECT queries are allowed for security reasons"}
    
    # Add LIMIT if not present and limit is specified
    if limit > 0 and 'LIMIT' not in sql_query.upper():
        sql_query = f"{sql_query} LIMIT {limit}"
    
    conn = _get_db_connection()
    cursor = conn.cursor()
    
    try:
        cursor.execute(sql_query)
        columns = [description[0] for description in cursor.description]
        rows = cursor.fetchall()
        
        # Convert to list of dictionaries for better readability
        results = []
        for row in rows:
            results.append(dict(zip(columns, row)))
            
        conn.close()
        return {
            "query": sql_query,
            "columns": columns,
            "results": results,
            "row_count": len(results)
        }
    except Exception as e:
        conn.close()
        return {"error": str(e), "query": sql_query}


@function_tool 
def get_sample_data(table_name: str, limit: int = 5) -> Dict[str, Any]:
    """Get sample rows from a specific table."""
    return execute_sql_query(f"SELECT * FROM {table_name}", limit=limit)

print("✅ Database tools defined successfully!")

✅ Database tools defined successfully!


## Create the SQL Agent with OpenAI Agents SDK

In [9]:
sql_agent = Agent(
    name="SQL Database Assistant",
    instructions=(
        "You are an expert SQL database assistant designed to interact with a SQLite database containing sales data. "
        "You can help users query and analyze the database by:\n"
        "1. Understanding the database structure (tables and schemas)\n"
        "2. Writing and executing appropriate SQL queries\n"
        "3. Providing clear explanations of results\n"
        "4. Following these important guidelines:\n"
        "   - Always use SELECT statements only - no INSERT, UPDATE, DELETE, or DROP operations\n"
        "   - Limit query results to reasonable amounts (use LIMIT when appropriate)\n"
        "   - Query only the relevant columns needed to answer the question\n"
        "   - Double-check your queries before execution\n"
        "   - Include explanations of how you arrived at your answer\n"
        "   - If a question isn't database-related, politely indicate you can only help with database queries\n"
        "   - Use the database tools available to explore schema and execute queries\n"
        "Your responses should be in Markdown format and always include an 'Explanation:' section with the SQL query used."
    ),
    tools=[
        get_table_list, 
        get_table_schema, 
        execute_sql_query, 
        get_sample_data
    ],
    model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
)

print("✅ SQL Agent created successfully!")

✅ SQL Agent created successfully!


## Test the SQL Agent

Now let's test our SQL agent with some example queries to demonstrate how it works with the OpenAI Agents SDK.

In [10]:
async def explore_database():
    """Basic database exploration demo."""
    prompt = "What tables are available in the database and what's the structure of the sales_data table?"
    result = await Runner.run(sql_agent, input=prompt)
    print("🔍 Database Structure:")
    print(result.final_output)
    return result

# Run the exploration
await explore_database()

OPENAI_API_KEY is not set, skipping trace export


🔍 Database Structure:
### Available Tables in the Database:
The database contains the following tables:
- **albums**
- **artists**
- **customers**
- **employees**
- **genres**
- **invoices**
- **invoice_items**
- **media_types**
- **playlists**
- **playlist_track**
- **tracks**
- **all_states_history**
- **sales_data**

### Structure of the `sales_data` Table:
The `sales_data` table has the following columns:

| Column Name         | Data Type    | Description |
|---------------------|--------------|-------------|
| **ORDERNUMBER**     | BIGINT       | Order number associated with the sale. |
| **QUANTITYORDERED** | BIGINT       | Quantity of items ordered. |
| **PRICEEACH**       | FLOAT        | Price per item. |
| **ORDERLINENUMBER** | BIGINT       | Line number within the order. |
| **SALES**           | FLOAT        | Total sales amount for the transaction. |
| **ORDERDATE**       | TEXT         | Date of the order. |
| **STATUS**          | TEXT         | Status of the order (e.g

RunResult(input="What tables are available in the database and what's the structure of the sales_data table?", new_items=[ToolCallItem(agent=Agent(name='SQL Database Assistant', instructions="You are an expert SQL database assistant designed to interact with a SQLite database containing sales data. You can help users query and analyze the database by:\n1. Understanding the database structure (tables and schemas)\n2. Writing and executing appropriate SQL queries\n3. Providing clear explanations of results\n4. Following these important guidelines:\n   - Always use SELECT statements only - no INSERT, UPDATE, DELETE, or DROP operations\n   - Limit query results to reasonable amounts (use LIMIT when appropriate)\n   - Query only the relevant columns needed to answer the question\n   - Double-check your queries before execution\n   - Include explanations of how you arrived at your answer\n   - If a question isn't database-related, politely indicate you can only help with database queries\n  

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


In [11]:
async def analyze_classic_cars_sales():
    """Replicate the example from the original LangChain prompt."""
    prompt = "What were the top 10 countries by sales amount for Classic Cars in 2003?"
    result = await Runner.run(sql_agent, input=prompt)
    print("🚗 Classic Cars Sales Analysis (2003):")
    print(result.final_output)
    return result

# Run the analysis
await analyze_classic_cars_sales()

🚗 Classic Cars Sales Analysis (2003):
### Top 10 Countries by Sales Amount for Classic Cars in 2003

Here is the list of the top 10 countries along with their total sales amounts for "Classic Cars" in the year 2003:

| Country     | Total Sales (USD) |
|-------------|--------------------|
| USA         | $558,544.09       |
| France      | $141,226.40       |
| Spain       | $129,763.64       |
| Norway      | $121,138.30       |
| Australia   | $85,388.64        |
| Singapore   | $78,090.03        |
| UK          | $66,705.63        |
| Denmark     | $60,786.02        |
| Germany     | $55,493.15        |
| Italy       | $43,233.38        |

---

### Explanation:
1. **SQL Query Used:**
   ```sql
   SELECT COUNTRY, SUM(SALES) AS TOTAL_SALES 
   FROM sales_data 
   WHERE PRODUCTLINE = 'Classic Cars' 
     AND YEAR_ID = 2003 
   GROUP BY COUNTRY 
   ORDER BY TOTAL_SALES DESC 
   LIMIT 10;
   ```
   - **`PRODUCTLINE = 'Classic Cars'`** filters data related to "Classic Cars."
   - **`YEAR_

RunResult(input='What were the top 10 countries by sales amount for Classic Cars in 2003?', new_items=[ToolCallItem(agent=Agent(name='SQL Database Assistant', instructions="You are an expert SQL database assistant designed to interact with a SQLite database containing sales data. You can help users query and analyze the database by:\n1. Understanding the database structure (tables and schemas)\n2. Writing and executing appropriate SQL queries\n3. Providing clear explanations of results\n4. Following these important guidelines:\n   - Always use SELECT statements only - no INSERT, UPDATE, DELETE, or DROP operations\n   - Limit query results to reasonable amounts (use LIMIT when appropriate)\n   - Query only the relevant columns needed to answer the question\n   - Double-check your queries before execution\n   - Include explanations of how you arrived at your answer\n   - If a question isn't database-related, politely indicate you can only help with database queries\n   - Use the database

In [12]:
async def custom_query(user_question: str):
    """Run a custom query using the SQL agent."""
    result = await Runner.run(sql_agent, input=user_question)
    print(f"📊 Query: {user_question}")
    print("Answer:")
    print(result.final_output)
    return result

# Example custom queries you can try:
# await custom_query("Which product line has the highest average order value?")
# await custom_query("Show me the top 5 customers by total sales amount")
# await custom_query("What are the sales trends by quarter in 2004?")

# Run an example query
await custom_query("Which product line has the highest total sales?")

📊 Query: Which product line has the highest total sales?
Answer:
The product line with the highest total sales is **Classic Cars**, with total sales amounting to **3,919,615.66**.

### Explanation:
I executed the following SQL query:

```sql
SELECT PRODUCTLINE, SUM(SALES) AS TOTAL_SALES 
FROM sales_data 
GROUP BY PRODUCTLINE 
ORDER BY TOTAL_SALES DESC 
LIMIT 1;
```

- **`SUM(SALES)`** aggregates the sales amounts for each product line.
- **`GROUP BY PRODUCTLINE`** organizes the sales data by product line.
- **`ORDER BY TOTAL_SALES DESC`** sorts the total sales in descending order.
- **`LIMIT 1`** ensures only the product line with the highest sales is returned.


RunResult(input='Which product line has the highest total sales?', new_items=[ToolCallItem(agent=Agent(name='SQL Database Assistant', instructions="You are an expert SQL database assistant designed to interact with a SQLite database containing sales data. You can help users query and analyze the database by:\n1. Understanding the database structure (tables and schemas)\n2. Writing and executing appropriate SQL queries\n3. Providing clear explanations of results\n4. Following these important guidelines:\n   - Always use SELECT statements only - no INSERT, UPDATE, DELETE, or DROP operations\n   - Limit query results to reasonable amounts (use LIMIT when appropriate)\n   - Query only the relevant columns needed to answer the question\n   - Double-check your queries before execution\n   - Include explanations of how you arrived at your answer\n   - If a question isn't database-related, politely indicate you can only help with database queries\n   - Use the database tools available to explo